In [19]:
import ast
import pandas as pd
import os
from itertools import product

performance_matrix_path = "./prediction_res/performance_matrix.csv"
power_cap_res_path = "./ecp_power_cap_res/2_dual_cap/"

# Applications of interest
apps = ['miniGAN', 'sw4lite', 'bert_large', 'UNet', 'Resnet50', 'lammps', 'gromacs', 'NAMD']
# apps = ['bert_large']

# Load performance matrix
performance_matrix = pd.read_csv(performance_matrix_path, index_col=0)

# Dictionary to store results
edp_results = {}

cpu_caps = [200, 190, 180, 170, 160, 150, 140]
gpu_caps = [250, 240, 230, 220, 210, 200, 190, 180, 170, 160, 150]
power_cap_pairs = list(product(cpu_caps, gpu_caps))

for app in apps:
    app_path = os.path.join(power_cap_res_path, app)
    

    min_edp = float('inf')
    best_power_pair = None

    for power_pair in power_cap_pairs:
        cpu_power, gpu_power = power_pair  # Unpack tuple

        # # Ensure the power pair exists in the performance matrix
        # if power_pair not in performance_matrix.index: 
        #     print(power_pair)
            
        # if app not in performance_matrix.columns:
        #     print(app)
            

        # Get normalized performance
        power_pair = f"({cpu_power}, {gpu_power})"

        norm_performance = performance_matrix.loc[power_pair, app]
        # print(norm_performance)

        # Compute normalized execution time
        norm_exec_time = 1 / norm_performance

        # Load CPU and GPU power files
        cpu_power_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_cpu_power.csv")
        gpu_power_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_gpu_metrics.csv")

   

        cpu_data = pd.read_csv(cpu_power_file)
        gpu_data = pd.read_csv(gpu_power_file)

        # Compute CPU energy consumption
        cpu_energy = (cpu_data["Package Power (W)"].mean() + cpu_data["DRAM Power (W)"].mean()) * norm_exec_time

        # Compute GPU energy consumption
        gpu_energy = gpu_data["Power (W)"].mean() * norm_exec_time

        
        # Total energy consumption
        total_energy = cpu_energy + gpu_energy
        

        # Compute EDP
        edp = total_energy * norm_exec_time

        # Update best power pair
        if edp < min_edp:
            min_edp = edp
            best_power_pair = (cpu_power, gpu_power)

        

    # Step 3: Compute EDP for actual performance using the selected power cap pair and baseline (200_250)
    if best_power_pair:
        cpu_power, gpu_power = best_power_pair

        # Baseline power cap (200_250)
        baseline_cpu_file = os.path.join(app_path, "200_250_cpu_power.csv")
        baseline_gpu_file = os.path.join(app_path, "200_250_gpu_metrics.csv")

    
        baseline_cpu_data = pd.read_csv(baseline_cpu_file)
        baseline_gpu_data = pd.read_csv(baseline_gpu_file)

        # Execution time from the last row of Time (s)
        baseline_exec_time = baseline_cpu_data["Time (s)"].iloc[-1]

        # Compute CPU energy for baseline
        baseline_cpu_energy = (baseline_cpu_data["Package Power (W)"].mean() + 
                               baseline_cpu_data["DRAM Power (W)"].mean()) * baseline_exec_time

        # Compute GPU energy for baseline
        baseline_gpu_energy = baseline_gpu_data["Power (W)"].mean() * baseline_exec_time

        # Total energy for baseline
        baseline_total_energy = baseline_cpu_energy + baseline_gpu_energy

        # Compute baseline EDP
        baseline_edp = baseline_total_energy * baseline_exec_time

        

        # Compute EDP for selected power cap pair
        selected_cpu_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_cpu_power.csv")
        selected_gpu_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_gpu_metrics.csv")

        if os.path.exists(selected_cpu_file) and os.path.exists(selected_gpu_file):
            selected_cpu_data = pd.read_csv(selected_cpu_file)
            selected_gpu_data = pd.read_csv(selected_gpu_file)

            # Execution time from the last row of Time (s)
            selected_exec_time = selected_cpu_data["Time (s)"].iloc[-1]

            # Compute CPU energy for selected power cap
            selected_cpu_energy = (selected_cpu_data["Package Power (W)"].mean() + 
                                   selected_cpu_data["DRAM Power (W)"].mean()) * selected_exec_time

            # Compute GPU energy for selected power cap
            selected_gpu_energy = selected_gpu_data["Power (W)"].mean() * selected_exec_time

            # Total energy for selected power cap
            selected_total_energy = selected_cpu_energy + selected_gpu_energy

            # Compute EDP for selected power cap
            selected_edp = selected_total_energy * selected_exec_time

            # Store results
            edp_results[app] = {
                "Best Power Pair": f"{cpu_power}_{gpu_power}",
                "Baseline EDP": baseline_edp,
                "Selected EDP": selected_edp
            }

In [20]:
# Calculate EDP savings for each application
for app, results in edp_results.items():
    baseline_edp = results["Baseline EDP"]
    selected_edp = results["Selected EDP"]
    
    # Compute EDP saving percentage
    edp_saving = ((baseline_edp - selected_edp) / baseline_edp) * 100

    # Print results with EDP savings
    print(f"Application: {app}")
    print(f"Best Power Pair: {results['Best Power Pair']}")
    print(f"Baseline EDP: {baseline_edp:.4f}")
    print(f"Selected EDP: {selected_edp:.4f}")
    print(f"EDP Saving: {edp_saving:.2f}%")
    print("-" * 40)

Application: miniGAN
Best Power Pair: 200_250
Baseline EDP: 127246.6588
Selected EDP: 127246.6588
EDP Saving: 0.00%
----------------------------------------
Application: sw4lite
Best Power Pair: 180_190
Baseline EDP: 574362.5053
Selected EDP: 563098.5916
EDP Saving: 1.96%
----------------------------------------
Application: bert_large
Best Power Pair: 170_200
Baseline EDP: 15806318.8566
Selected EDP: 15026736.4027
EDP Saving: 4.93%
----------------------------------------
Application: UNet
Best Power Pair: 170_220
Baseline EDP: 765812.5028
Selected EDP: 780168.3467
EDP Saving: -1.87%
----------------------------------------
Application: Resnet50
Best Power Pair: 180_160
Baseline EDP: 2398980.2197
Selected EDP: 2204205.5547
EDP Saving: 8.12%
----------------------------------------
Application: lammps
Best Power Pair: 170_180
Baseline EDP: 5154659.4818
Selected EDP: 5340482.0821
EDP Saving: -3.60%
----------------------------------------
Application: gromacs
Best Power Pair: 170_180
Ba

In [17]:
# Dictionary to store results
energy_saving_results = {}
perf_threshold = 0.95

for app in apps:
    app_path = os.path.join(power_cap_res_path, app)
    
    if not os.path.exists(app_path):
        continue

    max_energy_saving = -float('inf')  # Track max energy saving
    best_power_pair = None

    # Load baseline data (200_250)
    baseline_cpu_file = os.path.join(app_path, "200_250_cpu_power.csv")
    baseline_gpu_file = os.path.join(app_path, "200_250_gpu_metrics.csv")

    if not os.path.exists(baseline_cpu_file) or not os.path.exists(baseline_gpu_file):
        continue

    baseline_cpu_data = pd.read_csv(baseline_cpu_file)
    baseline_gpu_data = pd.read_csv(baseline_gpu_file)

    # Baseline execution time from the last row of Time (s)
    baseline_exec_time = baseline_cpu_data["Time (s)"].iloc[-1]

    # Compute baseline CPU and GPU energy consumption
    baseline_cpu_energy = (baseline_cpu_data["Package Power (W)"].mean() + 
                           baseline_cpu_data["DRAM Power (W)"].mean()) * baseline_exec_time

    baseline_gpu_energy = baseline_gpu_data["Power (W)"].mean() * baseline_exec_time

    # Total baseline energy consumption
    baseline_total_energy = baseline_cpu_energy + baseline_gpu_energy

    for cpu_power, gpu_power in power_cap_pairs:
        power_pair = f"({cpu_power}, {gpu_power})"  # Convert to string format

        # Ensure power pair exists and meets the performance threshold (≥ 0.95)
        if power_pair not in performance_matrix.index or app not in performance_matrix.columns:
            continue

        norm_performance = performance_matrix.loc[power_pair, app]
        if norm_performance < perf_threshold:
            continue  # Skip if performance loss >5%

        # Compute actual execution time
        norm_exec_time = 1 / norm_performance

        # Load actual CPU and GPU power data
        selected_cpu_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_cpu_power.csv")
        selected_gpu_file = os.path.join(app_path, f"{cpu_power}_{gpu_power}_gpu_metrics.csv")

        if not os.path.exists(selected_cpu_file) or not os.path.exists(selected_gpu_file):
            continue

        selected_cpu_data = pd.read_csv(selected_cpu_file)
        selected_gpu_data = pd.read_csv(selected_gpu_file)

        # Execution time from the last row of Time (s)
        selected_exec_time = selected_cpu_data["Time (s)"].iloc[-1]

        # Compute CPU and GPU energy consumption
        selected_cpu_energy = (selected_cpu_data["Package Power (W)"].mean() + 
                               selected_cpu_data["DRAM Power (W)"].mean()) * selected_exec_time

        selected_gpu_energy = selected_gpu_data["Power (W)"].mean() * selected_exec_time

        # Total selected energy consumption
        selected_total_energy = selected_cpu_energy + selected_gpu_energy

        # Compute energy savings percentage
        energy_saving = ((baseline_total_energy - selected_total_energy) / baseline_total_energy) * 100

        # Find the power cap pair with the largest energy saving
        if energy_saving > max_energy_saving:
            max_energy_saving = energy_saving
            best_power_pair = (cpu_power, gpu_power)
            best_exec_time = selected_exec_time
            best_energy = selected_total_energy

    # Store results
    if best_power_pair:
        energy_saving_results[app] = {
            "Best Power Pair": f"{best_power_pair[0]}_{best_power_pair[1]}",
            "Baseline Execution Time (s)": baseline_exec_time,
            "Selected Execution Time (s)": best_exec_time,
            "Runtime Overhead (%)": ((best_exec_time - baseline_exec_time) / baseline_exec_time) * 100,
            "Baseline Energy (J)": baseline_total_energy,
            "Selected Energy (J)": best_energy,
            "Energy Saving (%)": max_energy_saving
        }


In [18]:
# Print energy saving results
for app, results in energy_saving_results.items():
    print(f"Application: {app}")
    print(f"Best Power Pair: {results['Best Power Pair']}")
    # print(f"Baseline Execution Time (s): {results['Baseline Execution Time (s)']:.4f}")
    # print(f"Selected Execution Time (s): {results['Selected Execution Time (s)']:.4f}")
    print(f"Runtime Overhead (%): {results['Runtime Overhead (%)']:.2f}%")
    # print(f"Baseline Energy (J): {results['Baseline Energy (J)']:.2f}")
    # print(f"Selected Energy (J): {results['Selected Energy (J)']:.2f}")
    print(f"Energy Saving (%): {results['Energy Saving (%)']:.2f}%")
    print("-" * 40)


Application: miniGAN
Best Power Pair: 200_250
Runtime Overhead (%): 0.00%
Energy Saving (%): 0.00%
----------------------------------------
Application: sw4lite
Best Power Pair: 180_190
Runtime Overhead (%): 2.04%
Energy Saving (%): 3.92%
----------------------------------------
Application: bert_large
Best Power Pair: 160_210
Runtime Overhead (%): 5.89%
Energy Saving (%): 11.68%
----------------------------------------
Application: UNet
Best Power Pair: 180_210
Runtime Overhead (%): 6.51%
Energy Saving (%): 6.50%
----------------------------------------
Application: Resnet50
Best Power Pair: 180_180
Runtime Overhead (%): 6.00%
Energy Saving (%): 13.11%
----------------------------------------
Application: lammps
Best Power Pair: 170_150
Runtime Overhead (%): 7.56%
Energy Saving (%): 4.79%
----------------------------------------
Application: gromacs
Best Power Pair: 170_180
Runtime Overhead (%): 1.51%
Energy Saving (%): 8.00%
----------------------------------------
Application: NAMD
